In [1]:
import re as regex
from difflib import SequenceMatcher

In [2]:
class date:
    '''read and write dates in different formats'''

    def __init__(self,input_date,check=False):
        '''read in the input and check if its valid
        
        Parameters
        ----------
        can be either of 
        string : a valid date string in the gedcom format, 
                 e.g. "1 JAN 2000", or a date in the form
                 "yyyy.mm.dd" (legal seperators are ".","," and "/".
        tuple : a tuple of the form (yyyy,mm,dd)
        '''
        
        # determine what input is given and save it
        if isinstance(input_date,tuple):
            self.year,self.month,self.day = input_date
        else:
            self.string = input_date
            self.__parse(input_date)
        
        # for compatibility reasons
        self.tag   = 'DATE'
        self.value = self.print()
        
        if check:
            self.__consistency_check()
        
    def __parse(self,string):
        '''extract information from a date string'''
        
        abbreviations = ['BEF','AFT','CAL','EST','ABT','FROM','TO']
        months = {'JAN':1,'FEB':2,'MAR':3,'APR':4,'MAY':5,'JUN':6,
                  'JUL':7,'AUG':8,'SEP':9,'OCT':10,'NOV':11,'DEC':12}

        # define regex syntax for date format
        gedcom_syntax = ( '('+'|'.join(abbreviations)+')?\s?(\d*)?\s?(' +
                          '|'.join(months.keys()) + ')?\s?(\d*)\s?(.*)?' )
        free_syntax = '\(?([0-9?]+)[\./,]([0-9?]+)[\./,]([0-9?]+)\)?'

        if regex.match(free_syntax, string):
            year,month,day = regex.match(free_syntax, string).groups()
            try:
                self.year = int(year)
            except:
                pass
            try:
                self.month = int(month)
            except:
                pass
            try: 
                self.day = int(day)
            except:
                pass
                
        elif regex.match(gedcom_syntax, string):
            prefix,day,month,year, suffix = regex.match(gedcom_syntax, string).groups()   
            if prefix:
                self.prefix = prefix
            if year:
                self.year = int(year)
            if month:
                self.month = months[month]
            if not year and day:
                self.year = int(day)
            elif day:
                self.day = int(day)
            if suffix:
                self.suffix = suffix
        else:
            self.string = string
    
    def __consistency_check(self):
        '''make a simple consistency check for the entered date '''
    
        if hasattr(self,'year'):
            if self.year < 1000 or self.year > 2050:
                print(f'are you sure the year {self.string} is correct?')
        
        if hasattr(self,'month'):
            if self.month > 12:
                print(f'invalid value for month: {self.string}')
                
        if hasattr(self,'day'):
            if self.day > 31:
                print(f'invalid value for day: {self.string}')
        
        return
        
    def print(self,style='yyyy.mm.dd',separator='.'):
        '''
        legal styles
        ------------
        yyyy – four-digit year, e.g. 1996
        m – one-digit month for months below 10, e.g. 4
        mm – two-digit month, e.g. 04
        mmm – three-letter abbreviation for month, e.g. Apr
        mmmm – month spelled out in full, e.g. April
        d – one-digit day of the month for days below 10, e.g. 2
        dd – two-digit day of the month, e.g. 02


        not allowed 
        -----------
        yy – two-digit year, e.g. 96
        ddd – three-letter abbreviation for day of the week, e.g. Tue
        dddd – day of the week spelled out in full, e.g. Tuesday
        '''
    
        mmm = {1: 'Jan.',2: 'Feb.',3: 'März',4: 'Apr.',5: 'Mai',6: 'Jun.',
             7: 'Jul.',8: 'Aug.',9: 'Sept.',10: 'Okt.',11: 'Nov.',12: 'Dez.'}

        mmmm = {1: 'Januar',2: 'Februar',3: 'März',4: 'April',5: 'Mai',6: 'Juni',
            7: 'Juli',8: 'August',9: 'September',10: 'Oktober',11: 'November',12: 'Dezember'}

        prefix = {'BEF': 'vor','AFT': 'nach', 'CAL': 'errechnet',
            'EST': 'geschätzt', 'ABT': 'um', 'FROM': 'ab', 'TO': 'bis'}
        
        out = ''
        if hasattr(self,'prefix'):
            out += prefix[getattr(self,'prefix')] + ' '
        
        for component in style.split('.'):
            
            if component == 'yyyy':
                if hasattr(self,'year'):
                    out += f'{self.year}{separator}'
                #else:
                #    out += f'yyyy{separator}'
            elif component == 'm':
                if hasattr(self,'month'):
                    out += f'{self.month}{separator}'
                #else:
                #    out += f'm{separator}'
            elif component == 'mm':
                if hasattr(self,'month'):
                    out += f'{self.month:02d}{separator}'
                #else:
                #    out += f'mm{separator}'
            elif component == 'mmm':
                if hasattr(self,'month'):
                    out += f' {mmm[self.month]} '
                #else:
                #    out += f'mmm{separator}'
            elif component == 'mmmm':
                if hasattr(self,'month'):
                    out += f' {mmmm[self.month]} '
                #else:
                #    out += f'mmmm{separator}'
            elif component == 'd':
                if hasattr(self,'day'):
                    out += f'{self.day}{separator}'
                #else:
                #    out += f'd{separator}'
            elif component == 'dd':
                if hasattr(self,'day'):
                    out += f'{self.day:02d}{separator}'
                #else:
                #    out += f'dd{separator}'
            else:
                raise ValueError(f'Illigal format {component}')
                
        return out.strip(' ').strip('.')
    
    
    def __str__(self):
        '''inside of print provide only selected information'''
        
        return self.print()
        
    def __repr__(self):
        '''outside of print provide as many information as possible'''
        
        return self.print()


In [3]:
class Attribute:
    '''save line from gedcom file

    a gedcom file consists of lines with the following structure
    
    
      level [pointer] tag [value]

    [] are optional arguments that are only saved when available
    
    level   : hierarchy of the entry
    pointer : unique identifier for this entry
    tag     : type of entry (eg.g INDI, FAM, SEX, NAME, etc.)
    value   : data in this entry
    '''
    
    def __init__(self,level,pointer,tag,value):

        self.level   = level
        self.tag     = tag
        self.entries = []
        
        if pointer:
            self.pointer = pointer
        if value:
            self.value = value

    def __str__(self):
        '''inside of print provide only selected information'''
        
        if hasattr(self,'value'):
            return self.value
        else:
            return ( self.tag + ': ' 
                   + getattr(self,'pointer','')
                   + ' (' + ', '.join(getattr(self,'entries',[])) + ')'
                   )
        
    def __repr__(self):
        '''outside of print provide as many information as possible'''
        
        return ( self.tag + ': ' 
               + getattr(self,'pointer','')
               + getattr(self,'value','') 
               + ' (' + ', '.join(getattr(self,'entries',[])) + ')'
               )

    def clean(self):
        delattr(self,'level')
        if hasattr(self,'parent'):
            delattr(self,'parent')
        for k,v in self.__dict__.items():
            if isinstance(v,Attribute):
                v.clean()

    def info(self):
        '''print out all information saved in attribute'''
        
        if self.entries:
            tag   = getattr(self,'tag')
            value = getattr(self,'value','')
            print(f'{tag}: {value}')
            
            for entry in self.entries:
                att = getattr(self,entry)
                att.info()
        else:
            tag   = getattr(self,'tag')
            value = getattr(self,'value','')
            print(f'{tag}: {value}')
                

In [19]:
class Gedcom:
    '''Read in gedcom files and save in an easy to use structure
    
    
    from https://github.com/madprime/python-gedcom/blob/master/gedcom/__init__.py
    
    '''

    # ===========================================================
    # initialize the data structure and read data from file
    # ===========================================================
    
    def __init__(self,file):
        
        self.Elements = {}
        
        # read in the gedcom file (with utf8 enconding)
        with open(file,'r',encoding="utf-8") as f:
            raw = f.read()
            
        lines = raw.strip('\ufeff').split('\n')

        # go over all lines in file and save data
        for line_num, line in enumerate(lines[:-2]):
            # each line can contain up to 4 fields
            level, pointer, tag, value = Gedcom.__split(line,line_num)         
            
            if level == 0:    
                # level == 0 is an Element with additional methods to retrive informations
                element = Attribute(level,pointer,tag,value)
                self.Elements[pointer] = element
                last_element = element
                
            else: 
                # level >= 1 does not include those methods (otherwise identical)
                element = Attribute(level,pointer,tag,value)
                parent_element = last_element
                while parent_element.level > level-1:
                    parent_element = parent_element.parent
                
                # CONT is appended to the previous attribute
                if tag == 'CONT':
                    if hasattr(parent_element,'value'):
                        setattr(parent_element,'value',getattr(parent_element,'value') + '\n' + value)
                    else:
                        setattr(parent_element,'value','\n ' + value)
                    continue
                    
                element.parent = parent_element
                
                # avoid two entries with the same tag
                if not getattr(parent_element,tag,False):
                    setattr(parent_element,tag,element)
                    parent_element.entries.append(tag)
                else:
                    i = 1
                    while True:
                        if not getattr(parent_element,tag+str(i),False):
                            setattr(parent_element,tag+str(i),element)
                            parent_element.entries.append(tag+str(i))
                            break
                        else:
                            i+=1
                            
                last_element = element
        
        # the last thing we do is clean up temporarily data
        for k,v in self.Elements.items():
            v.clean()

        # create index (list of people) for search
        self.people = []
        for k,v in self.Elements.items():
            if v.tag == 'INDI':
                name_date = self.print_name(k)
                name = name_date[:name_date.find('[')-1]
                date = name_date[name_date.find('['):]
                self.people.append((k,name,date))
           
        people = filter(regex.compile('@I\d*@').match, self.Elements.keys())
        families = filter(regex.compile('@F\d*@').match, self.Elements.keys())
        sources = filter(regex.compile('@S\d*@').match, self.Elements.keys())
        
        print(f'File "{file}" was successfully imported\n' + 
              f' - {len(list(people))} people\n' +
              f' - {len(list(families))} families\n' +
              f' - {len(list(sources))} sources'
             )
        
    @staticmethod
    def __split(line,line_num):
        '''split a gedcom line into level, tag and data
        
        '''
        
        ged_line_regex = (
            # Level must start with nonnegative int, no leading zeros.
                '^(0|[1-9]+[0-9]*) ' +
                # Pointer optional, if it exists it must be flanked by '@'
                '(@[^@]+@ |)' +
                # Tag must be alphanumeric string
                '([A-Za-z0-9_]+)' +
                # Value optional, consists of anything after a space to end of line
                '( [^\n\r]*|)' 
                # End of line defined by \n or \r
                #'([\r\n]{1,2})'
        )
        if regex.match(ged_line_regex, line):
            line_parts = regex.match(ged_line_regex, line).groups()
        else:
            error_message = (f"Line {line_num} of document violates GEDCOM format\n {line}" +
                             "\nSee: http://homepages.rootsweb.ancestry.com/" +
                             "~pmcbride/gedcom/55gctoc.htm\n")
            raise SyntaxError(error_message)
        
        level = int(line_parts[0])
        pointer = line_parts[1].rstrip(' ')
        tag = line_parts[2]
        value = line_parts[3][1:]
        
        return level, pointer, tag, value   

    # ===========================================================
    # general functions that help navigate the data structure
    # ===========================================================
    
    def find_by_name(self,search_name):
        ''' search record by name''' 

        for ID,name,date in self.people:
            if SequenceMatcher(None, search_name, name).ratio() > .8:
                print(f'{ID:>8}: {name} {date}')
    
    def get_element(self,pointer):
        '''return attribute element to pointer if it exists'''
        
        if pointer in self.Elements:
            return self.Elements[pointer]
        elif f'@{pointer}@' in self.Elements:
            return self.Elements[f'@{pointer}@']
        else:
            raise KeyError(f'no Element {pointer} in file')

    def hasattr(self,pointer,attribute):
        '''check if element pointer has attribute
        
        Parameters
        ----------
        
        pointer : pointer to the element
        attribute : Name of the attribute (can be nested with ".")
        
        The retrieving is done via the auxiliary function __getattr()
        '''
        
        element = self.get_element(pointer)
        attribute = attribute.split('.')
        
        return self.__hasattr(element,attribute)

    def __hasattr(self,element,attribute):
        '''auxiliary function to hasattr (for recursion)'''
        
        level0 = attribute.pop(0)
        
        if hasattr(element,level0):
            element = getattr(element,level0)
        else:
            return False
        
        if attribute:
            return self.__hasattr(element,attribute)
        else:
            if hasattr(element,'value'):
                return True
            else:
                return False
    
    def getattr(self,pointer,attribute,default=None):
        '''retrieve attribute from element pointer
        
        This function tries to retrieve the attribute from the element.
        designated by pointer. It raises an error if element does not 
        have such an attribute. To avoid this, check beforehand if the
        attribute exists with hasattr().
        
        Parameters
        ----------
        
        pointer : pointer to the element
        attribute : Name of the attribute (can be nested with ".")
        default : if a default value is passed no error is raised 
                 in case the attribute does not exist
        
        The retrieving is done via the auxiliary function __getattr()
        '''
        
        element = self.get_element(pointer)
        attribute = attribute.split('.')
    
        return self.__getattr(element,attribute,default=default)
    
    def __getattr(self,element,attribute,default=None):
        '''auxiliary function to getattr (for recursion)'''
        
        level0 = attribute.pop(0)
        
        if hasattr(element,level0):
            element = getattr(element,level0)
        elif default is not None:
            return default
        else:
            error_message = (f"{element.tag} has no attribute {level0} (try " + 
                             ', '.join(getattr(element,'entries',['...'])) + ')')       
            raise AttributeError(error_message)

        if attribute:
            return self.__getattr(element,attribute,default=default)
        else:
            if hasattr(element,'value'):
                return getattr(element,'value')
            elif default is not None:
                return default
            else:
                error_message = (f"{element.tag} has no attribute value (try" + 
                                 ', '.join(getattr(element,'entries',['...'])) +')' )       
                raise AttributeError(error_message)
        
    def __sort_by_birth(self,lst):
        '''Sort the given list by their birtday
        
        Paramters
        ---------
        lst : a list of pointers
        
        '''
        
        return sorted(lst,key = lambda x: date(self.getattr(x,'BIRT.DATE',default='9999')).print('yyyy'))

        
    # ===========================================================
    # specific functions that print out one formated field 
    # ===========================================================

    def print_event(self,pointer,event):
        '''print out formatted string for different events
        
        An event is any attribute of the form event: date in place.
        The following events are supported
        
        Geburt:     BIRT
        Taufe :     CHR
        Hochzeit:   not yet supported
        Tod :       DEAT
        Beerdigung: BURI
        '''
        
        if event == 'MARR' and self.get_element(pointer).tag == 'INDI':
            
            marriage = ''
            count = 0
            for i in self.get_element(pointer).entries:
                if i.startswith('FAMS'):
                    
                    if count >= 1:
                        marriage += '\\\\\n & '
                    
                    famID = self.getattr(pointer,i)
                    if self.hasattr(famID,'HUSB'):
                        HUSB = self.getattr(famID,'HUSB')
                    else:
                        HUSB = None
                    if self.hasattr(famID,'WIFE'):
                        WIFE = self.getattr(famID,'WIFE')
                    else:
                        WIFE = None
                    marriage += self.print_event(famID,'MARR')
                    
                    if pointer != HUSB and HUSB:
                        marriage += f' mit {self.print_name(HUSB,short=True)}'
                    elif pointer != WIFE and WIFE:
                        marriage += f' mit {self.print_name(WIFE,short=True)}'
                    
                    count += 1
                    
            return marriage
                    
        out = ''
        if self.hasattr(pointer,f'{event}.DATE'):
            d = date(self.getattr(pointer,f'{event}.DATE'))
            out += d.print('dd.mmmm.yyyy')

        if self.hasattr(pointer,f'{event}.PLAC'):
            place = self.getattr(pointer,f'{event}.PLAC')
            out += f' in {place}'
        
        return out
    
    def print_name(self,pointer,short=False,link=False):
        '''prints out name together with date of birth and death'''
        
        if self.hasattr(pointer,'BIRT.DATE'):
            d1 = date(self.getattr(pointer,'BIRT.DATE')).print(style='dd.mm.yyyy')
        else:
            d1 = ''
        if self.hasattr(pointer,'DEAT.DATE'):
            d2 = date(self.getattr(pointer,'DEAT.DATE')).print(style='dd.mm.yyyy')
        else:
            d2 = ''
        name = ''
        if self.hasattr(pointer,'NAME.GIVN'):
            name += self.getattr(pointer,'NAME.GIVN') + ' '
        if self.hasattr(pointer,'NAME.SURN'):
            name += self.getattr(pointer,'NAME.SURN')
        
        if link:
            out = f'\hyperref[{pointer}]{{{name}}} '
        else:
            out = f'{name} '
        
        # if option short is used we don't add the birthdate etc.
        if short:
            return out
        
        if d1: 
            out += f'[{d1}'
        else:
            out += '[...'
        if d2: 
            out += f'--{d2}'
        
        children = self.get_children(pointer)
        if len(children)==1:
            out += f' ({len(children)} Kind)]'
        elif len(children) > 1:
            out += f' ({len(children)} Kinder)]'      
        else:
            out += ']'
            
        return out
    
    def get_father(self,pointer):
        '''return pointer to father'''
        
        if self.hasattr(pointer,'FAMC'):
            famID = self.getattr(pointer,'FAMC')
        else:
            return None
        if self.hasattr(famID,'HUSB'):
            return self.getattr(famID,'HUSB')
        else:
            return None
        
    def get_mother(self,pointer):
        '''return pointer to mother'''
        
        if self.hasattr(pointer,'FAMC'):
            famID = self.getattr(pointer,'FAMC')
        else:
            return None
        if self.hasattr(famID,'WIFE'):
            return self.getattr(famID,'WIFE')
        else:
            return None
   
    def get_children(self,pointer):
  
        element = self.get_element(pointer)

        children = []
        for i in element.entries:
            if i.startswith('FAMS'):
                famID = self.getattr(pointer,i)
                
                family = self.get_element(famID)
                for j in family.entries:
                    if j.startswith('CHIL'):
                        children.append(self.getattr(famID,j))
            
        return self.__sort_by_birth(children)
            
    def print_children(self,pointer):
        
        children = self.get_children(pointer)
        
        out = ''
        for child in children:
            out += self.print_name(child) + '\n'
        return out
        
    def get_siblings(self,pointer):
        
        siblings = set()
        father = self.get_father(pointer)
        if father:
            siblings.update(self.get_children(father))
        mother = self.get_mother(pointer)
        if mother:
            siblings.update(self.get_children(mother))   
        
        if not siblings:
            return []
        
        try:
            siblings.remove(pointer)
        except:
            siblings.remove(f'@{pointer}@')
            
        return self.__sort_by_birth(list(siblings))
        
    def get_sources(self,pointer):
        
        sources = []
        for i in self.get_element(pointer).entries:
            if i.startswith('SOUR'):
                source = ''
                s = self.getattr(pointer,i)
                
                # if source points to something 
                if regex.match('@S\d*@', s):
                    source += self.getattr(s,'TITL')
                    if self.hasattr(pointer,f'{i}.PAGE'):
                        source += ', ' + self.getattr(pointer,f'{i}.PAGE')
                        
                    if self.hasattr(pointer,f'{i}._LINK'):
                        link = self.getattr(pointer,f'{i}._LINK')
                        source = f'\\href{{{link}}}{{{source}}}'
                else:
                    source = s
                
                    # replace special characters in LaTeX
                    source = source.replace('&','\\&')
                    source = source.replace('%','\\%')
                
                sources.append(source)
        
        return sources
            
    def create_personenblatt(self,pointers):
        '''create the source code for a Personenblatt in LaTeX''' 
        
        # make sure pointers is a list
        if not isinstance(pointers, list):
            pointers = [pointers]
        
        txt = ''
        
        for pointer in pointers:
            
            # we skip lines with strings
            if not regex.match('@I\d*@', pointer):
                txt += '\n' + pointer + '\n\n'
                continue
            
            # first we nee some basic information
            if self.hasattr(pointer,'NAME.SURN'):
                surname = self.getattr(pointer,'NAME.SURN')
            else:
                surname = ''
            if self.hasattr(pointer,'NAME.GIVN'):
                givenname = self.getattr(pointer,'NAME.GIVN')
            else:
                givenname = ''
            if self.hasattr(pointer,'BIRT.DATE'):
                d = date(self.getattr(pointer,'BIRT.DATE'))
                suffix = d.print(style='yyyy')
            else:
                suffix = ''
            if self.hasattr(pointer,'DEAT.DATE'):
                d = date(self.getattr(pointer,'DEAT.DATE'))
                suffix += f'--{d.print(style="yyyy")}'
            label = pointer
            if self.hasattr(pointer,'OBJE.FILE'):
                filename = self.getattr(pointer,'OBJE.FILE').split('\\')[-1].split('.')[0]
            else:
                filename = None

            # start with the person environment
            txt += f'''\\begin{{person}}[
    surname = {{{surname}}},
    givenname = {{{givenname}}},
    suffix = {{{suffix}}},
    label = {{{label}}}'''
            
            if filename:
                 txt += f',\n    filename = {{{filename}}}\n    ]\n\n'
            else:
                txt += '\n    ]\n\n'
        
            # next we create the table with events
            events = {'BIRT' : '\\geboren', 'CHR' : '\\taufe', 'MARR' : '\geheiratet', 'DEAT' : '\\gestorben', 'BURI' : '\\bestattet'}

            txt += '\\begin{tabular}{cl}\n'

            for k,v in events.items():
                event = self.print_event(pointer,k)
                if event:
                    txt += f'{v} & {event}\\\\\n'

            txt += '\\end{tabular}\\\\\n\\medbreak\n'

            # print parents
            father = self.get_father(pointer)
            mother = self.get_mother(pointer)
            if father: 
                if father in pointers:
                    txt += f'\\textsc{{vater}}: {self.print_name(father,link=True)}'
                else: 
                    txt += f'\\textsc{{vater}}: {self.print_name(father,link=False)}'
                if mother: 
                    txt += '\\\\\n'
            if mother: 
                if mother in pointers:
                    txt += f'\\textsc{{mutter}}: {self.print_name(mother,link=True)}\n'
                else: 
                    txt += f'\\textsc{{mutter}}: {self.print_name(mother,link=True)}\n'
            # empty line after parents
            if father or mother:
                txt += '\\medbreak\n'
               
            # next are siblings:
            siblings = self.get_siblings(pointer)
            if siblings:
                txt +='\\textsc{{geschwister}}\n\\begin{itemize}\n'
                for sibling in siblings:
                    if sibling in pointers:
                        txt += f'\\item {self.print_name(sibling,link=True)}\n'
                    else: 
                        txt += f'\\item {self.print_name(sibling,link=False)}\n'
                txt +='\\end{itemize}\n\\bigbreak\n'
                
            # next are siblings:
            childrens = self.get_children(pointer)
            if childrens:
                txt +='\\textsc{{kinder}}\n\\begin{itemize}\n'
                for child in childrens:
                    if child in pointers:
                        txt += f'\\item {self.print_name(child,link=True)}\n'
                    else: 
                        txt += f'\\item {self.print_name(child,link=False)}\n'
                txt +='\\end{itemize}\n\\medbreak\n'
            
            # Notes
            if self.hasattr(pointer,'NOTE'):
                txt +='\\textsc{anmerkung}\\\\\n'
                txt += self.getattr(pointer,'NOTE')
                txt += '\n\medbreak\n'
                
            # Quellen
            sources = self.get_sources(pointer)
            if sources:
                txt += '\\textsc{{quellen}}\n\\begin{enumerate}[label={[\\arabic*]}]\n' 
                for source in sources:
                    txt += '\\item ' + source + '\n'
                txt += '\\end{enumerate}\n\n'
            
            txt +='\\end{person}\n\n'

        #txt = txt.replace('ö','"o')
        #txt = txt.replace('ü','"u')
        #txt = txt.replace('ä','"a')
        #txt = txt.replace('ß','"s')
        
        with open('gedcom_to_LaTeX.tex','w',encoding="utf-8") as f:
            f.write(txt)

        print('file successfully created')
        return 0
        
    def build_family_tree(self,start,generations=3,siblings=False,chapters=False,save=False):
        '''Build family tree from gedcom file (list of pointers)
        
        More precise this fuction creates a list of pointers for an
        Ahnentafel starting from person with ID start.
        
        Parameters
        ----------
        
        start : pointer for the starting person
        generations : number of generations (e.g. 2 for parents only)
        siblings : include siblings for parents
        chapters : insert chapter marks to the list
        save : if true we call the function create_personenblatt to create a LaTeX file
        '''
        
        levels = {1: 'Eltern',2: 'Gro"seltern', 3:'Ur-Gro"seltern', 4: 'Ur-Ur-Gro"seltern', 
                  5:'Ur-Ur-Ur-Gro"seltern', 6:'Ur-Ur-Ur-Ur-Gro"seltern'}
        
        if start in self.Elements:
            pass
        elif f'@{start}@' in self.Elements:
            start = f'@{start}@'
        else:
            raise KeyError(f'no Element {pointer} in file')
        
        if generations < 2:
            raise ValueError('family tree must have at least 2 generations')
        
        tree = []
        previous = [start]
        
        for g in range(1,generations):
                
            if chapters:
                tree.append(f'\\addchap{{{levels[g]}}}\n')
            
            new = []
            for child in previous:
                father = self.get_father(child)
                mother = self.get_mother(child)
                
                if chapters:
                    tree.append(f'\\addsec{{{self.print_name(father,short=True)} \\& ' +
                                f'{self.print_name(mother,short=True)}}}\n')
                    
                new.append(father)
                new.append(mother)
                
                tree.append(father)
                tree.append(mother)
                
                if siblings:
                    tree += self.get_children(father)
                    tree += self.get_children(mother)
            previous = new
        
        people = []
        for person in tree:
            if person not in people:
                people.append(person)
                
        if save:
            self.create_personenblatt(people)
        else:
            return people
        
ged = Gedcom('stammbaum3.ged')

File "stammbaum3.ged" was successfully imported
 - 2121 people
 - 752 families
 - 98 sources


## use the build_family_tree method to create a LaTeX source file

In [20]:
ged.build_family_tree('@I1@',generations=6,chapters=True,siblings=True,save=True)

file successfully created


## play around

In [16]:
ged.get_element('I7').SOUR3._LINK

_LINK: https://www.familysearch.org/tree/person/details/L51R-B53 ()

In [219]:
ged.get_element('I951').info()

INDI: 
RIN: AB:I951
_UID: F098DA55069D460480416150CDBFAF30
NAME: Eva Barbara /Zeller/
SURN: Zeller
GIVN: Eva Barbara
SEX: F
FAMS: @F324@
NOTE: zweiter Vorname unleserlich. Geburtsname auf Eheurkunde des Johann Valentin könnte auch Zöller sein
SOUR: Eheurkunde Valentin Scheuermann
https://gw.geneanet.org/wzipp?lang=de&p=eva+barbara&n=zeller
BIRT: 
DATE: 3 DEC 1811
PLAC: Kirchzell
DEAT: 
DATE: 30 JAN 1886
PLAC: Ottorfszell
CHAN: 
DATE: 15 SEP 2018
TIME: 18:11:18


In [189]:
ged.print_name('I1',short=True)

'Fabian Scheuermann '

In [160]:
indi = ged.get_element('I180')
indi.NAME.NICK

NICK: Jus ()

In [172]:
lst = ged.get_siblings('I4')

for p in lst:
    print(ged.print_name(p))
    
lst = sorted(lst,key = lambda x: date(ged.getattr(x,'BIRT.DATE',default='9999')).print('yyyy'))

print('\n and now sorted \n')

for p in lst:
    print(ged.print_name(p))

Leo Galm [07.09.1927--24.05.2003 (4 Kinder)]
Georg Galm [16.08.1929 (3 Kinder)]
Maria Galm [14.07.1922--15.02.1995 (7 Kinder)]
Arthur Galm [02.10.1925--1944]
Philipp Galm [19.02.1920--vor 1945]
Juliane Galm [26.03.1921--05.04.2019 (5 Kinder)]

 and now sorted 

Philipp Galm [19.02.1920--vor 1945]
Juliane Galm [26.03.1921--05.04.2019 (5 Kinder)]
Maria Galm [14.07.1922--15.02.1995 (7 Kinder)]
Arthur Galm [02.10.1925--1944]
Leo Galm [07.09.1927--24.05.2003 (4 Kinder)]
Georg Galm [16.08.1929 (3 Kinder)]


In [180]:
date('9999').print('yyyy')

'9999'

In [155]:
ged.find_by_name('Julius Galm')

    @I7@: Julius Galm [28.02.1883--23.05.1929 (7 Kinder)]
  @I180@: Julius Galm [21.02.1857--03.01.1929 (4 Kinder)]
 @I1312@: Julius Galm [20.08.1890--nach 1932]


In [97]:
if regex.match('@S\d*@', s):
    print(ged.getattr(s,'TITL'))

Ahnentafel Galm


In [144]:
date?